In [ ]:
import os
import sys
from pathlib import Path
path = os.path.realpath('__file__')
parent_dir = str(Path(path).parents[1])
sys.path.append(parent_dir)
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# rational field
x = np.array([i/200.0 for i in range(-1000,1000,1)])
# y^2 = x^3 + 7 F_p
y = np.sqrt(x**3 + 7)
plt.scatter(x,y)
plt.scatter(x,-y)

In [ ]:
# finite field
x = np.array([i for i in range(0,1000000,5000)])
# y^2 = x^3 + 7 F_p
y = np.sqrt((x%251)**3 + 7) % 251
#G would be a generator.

In [ ]:
plt.scatter(x,y)
plt.scatter(x,-y)

In [ ]:
import zkbp
from pyledger.zkutils import Commit,r_blend

In [ ]:
# create a new blinding factor "r" between 1-n % q (256bits) 
r=r_blend()
r.to_str()

In [ ]:
# generate gh object, contains g, h point generators, constant for the ledger and public
gh = zkbp.gen_GH()

### Pedersen Commitment and homomorphicity in Tx

In [ ]:
# Now pedersen commit can then be:
ped_100 = Commit(gh,100,r)
ped_100.eval.get # = g^100 * h^r (in compressed form)

In [ ]:
# second pedersen commit can then be:
r2=r_blend()
ped_minus50 = Commit(gh,-60,r2)

In [ ]:
# third and summed to 0 sum_rs=0:
r3=-(r2+r)
(r+r2+r3).to_str()

In [ ]:
ped_50b = Commit(gh,-40,r3)

In [ ]:
sum_cms = ped_50b + ped_minus50 + ped_100 # actually means: (g^100*h^r1)*(g^-60*h^r2)*(g^-40*h^r3)=g^0*h^0=1.0
sum_cms.eval.get # results 0 here would mean g^0h^0

### Audit Token

In [ ]:
from pyledger.zkutils import Token

In [ ]:
# token here (h^sk)^r or (pk)^r
pbsk1 = zkbp.gen_pb_sk(gh)
tk1=Token(pbsk1,r)
tk1.eval.get # (pk)^r thats what we share

In [ ]:
# consenting and finding my value
cm1_enc = ped_100.eval.get
tk1_enc = tk1.eval.get

In [ ]:
# brutforce value using cm tk and my sk.
zkbp.get_brut_v(zkbp.from_str(cm1_enc),
                zkbp.to_token_from_str(tk1_enc),
                gh,pbsk1,1000)